In [1]:
from typing import List, Tuple, Optional, Generator

In [2]:
# интерпретатор фрактрана над разложениями - векторами степеней простых множителей-делителей

IntState = List[int]  # с целыми неотрицательными компонентами
FracState = List[int] # с целыми компонентами

Comment = str
Command = Tuple[FracState, Comment]

Program = List[Command]

Emission = Tuple[int, IntState]  # номер команды и следующее состояние


def is_int(state: FracState) -> bool:
    '''
    проверяет, является ли дробь целым (отсутствуют отрицательные степени)
    '''
    return all(v >= 0 for v in state)

def product(state: IntState, cmd: FracState) -> FracState:
    '''
    находит произведение состояния на команду 
    '''
    assert len(state) == len(cmd)
    return [(a + b) for (a, b) in zip(state, cmd)]

def try_step(state: IntState, program: Program) -> Optional[Emission]:
    '''
    пытается выполнить один такт программы,
    возвращает номер команды и новое состояние;
    если произошла остановка, возвращает None
    '''
    for i, (cmd, _) in enumerate(program):
        new_state = product(state, cmd)
        if is_int(new_state):
            return i, new_state
    return None

def run(state: IntState, program: Program) -> Generator[Emission, None, None]:
    '''
    последовательность номеров команд и новых состояний, до остановки
    '''
    while True:
        result = try_step(state, program)
        if not result:
            break
        yield result
        i, state = result

def run_until_stop(state: IntState, program: Program) -> IntState:
    '''
    выполняет программу до остановки и возвращает последнее состояние
    '''
    while True:
        result = try_step(state, program)
        if not result:
            return state
        i, state = result

In [3]:
# человекочитаемый генератор программы для задачи Коллаца

# избранные вершины
DIV, EVEN, ODD, START = 3, 2, 1, 0

# состояние машины - это вектор [x, n, y, v1, v2]
# где
# x - стартовое/текущее значение элемента последовательности,
# n - длина последовательности
# y - вспомогательная переменная (соответствует x/2)
# v1, v2 - номер вершины автомата (в двудольном графе)

# функции add_... возвращают подпрограммы, то есть, несколько команд подряд
# ...left - из левого подграфа в правый
# ...right - из правого в левый
# ...twice - симметричная пара

def concat(*progs : List[Program]) -> Program:
    return sum(progs, [])

def add_left(dx : int, dn : int, dy : int, v_from : int, v_to : int, c : Comment) -> Program:
    return [([dx, dn, dy, -v_from, +v_to], c)]


def add_right(dx : int, dn : int, dy : int, v_from : int, v_to : int, c : Comment) -> Program:
    return [([dx, dn, dy, +v_to, -v_from], c)]

def add_twice(dx : int, dn : int, dy : int, v_from : int, v_to : int, c : Comment) -> Program:
    return concat(
        add_left (dx, dn, dy, v_from, v_to, c),
        add_right(dx, dn, dy, v_from, v_to, c)
    )

def add_special(x : int) -> Program:
    '''
    подпрограмма обработки специального числа
    '''
    x2, x31 = x // 2, x * 3 + 1
    if x % 2:
        return concat(
            add_right(   -1, 0, x2, x, DIV  , f"x > {x}"     ),
            add_right(    0, 0,  0, x, x    , f"x = {x}"     ),
            add_left (  x31, 1,  0, x, START, f"x' = {x31}"  )
        )
    else:
        return concat(
            add_right(   -2, 0, x2, x, DIV  , f"x > {x}+1"   ),
            add_right(   -1, 0,  1, x, x    , f"x = {x}+1"   ),
            add_right(    0, 0,  0, x, x    , f"x = {x}"     ),
            add_left (x31+3, 1, -1, x, START, f"x' = {x31+3}"),
            add_left (x2   , 1,  0, x, START, f"x' = {x2}"   ),
        )

def add_specials(xs : List[int]) -> Program:
    '''
    все специальные числа
    '''
    return concat(*(add_special(x) for x in xs))

def add_div(factors = [1000, 100, 10, 1]) -> Program:
    '''
    вершина деления пополам
    '''
    return concat(
        *(add_twice(-f*2, 0, f, DIV, DIV, f"DIV x/2 ...") for f in factors),
        add_twice(-1, 0, 1, DIV, ODD , f"DIV odd x"),
        add_twice( 0, 0, 1, DIV, EVEN, f"DIV even x"),
    )

def add_even(factors = [1000, 100, 10, 1]) -> Program:
    '''
    вершина чётного числа
    '''
    return concat(
        *(add_twice(f, 0, -f, EVEN, EVEN, f"EVEN x/2 ...") for f in factors),
        add_twice(0, 1, 0, EVEN, START, f"EVEN x' = x/2"),
    )

def add_odd(factors = [1000, 100, 10, 1]) -> Program:
    '''
    вершина нечётного числа
    '''
    return concat(
        *(add_twice(f*6, 0, -f, ODD, ODD, f"ODD 3x+1 ...") for f in factors),
        add_twice(4, 1, 0, ODD, START, f"ODD x' = 3x+1"),
    )

def add_start_special(x : int) -> Program:
    '''
    переход к специальному числу
    '''
    return add_left(-x, 0, 0, START, x, f"START x >= {x}")

def add_start_specials(xs : List[int]) -> Program:
    '''
    переходы ко всем специальным числам
    '''
    return concat(*(add_start_special(x) for x in xs))

def add_start_common() -> Program:
    '''
    основная часть стартовой вершины
    '''
    return concat(
        add_left(-2, 0, 0, START, DIV, f"START x >= 2"),
        add_left(-1, 1, 0, START, START, f"START x = 1 FINISH"),
    )

def add_start(xs : List[int]) -> Program:
    '''
    стартовая вершина
    '''
    return concat(
        add_start_specials(xs),
        add_start_common(),
    )

def make_program(specials : List[int] = [], factors : List[int] = [1000, 100, 10, 1]) -> Program:
    '''
    программа целиком
    параметризуется двумя списками: специальные числа и факторы ускорения операций
    '''
    assert not specials or min(specials) > 3
    specials = sorted(set(specials or []), reverse=True)

    factors = sorted((set(factors or []) | {1}), reverse=True)  # 1 обязательно должен быть

    return concat(
        add_specials(specials),
        add_div(factors),
        add_even(factors),
        add_odd(factors),
        add_start(specials),
    )

def initial_state(x : int) -> IntState:
    return [x, 0, 0, START, 0]

def get_answer(state : IntState) -> int:
    x, n, y, v1, v2 = state
    assert (x, y, v1, v2) == (0, 0, 0, 0)
    return n

In [4]:
the_program = make_program(specials=[13, 16, 9232], factors=[1000, 500, 100, 50, 10, 5, 1])

len(the_program)

68

In [5]:
def run_kollatz(x : int, program) -> int:
    def print_head():
        print( 'шаг  |   x   |   n   |   y   |   v1  |   v2  | комментарий')
    def print_bar():
        print( '-----+-------+-------+-------+-------+-------+------------')
    def print_state(t, s : FracState, c : Comment):
        x, n, y, v1, v2 = s        
        print(f'{t:4} | {x:5} | {n:5} | {y:5} | {v1:5} | {v2:5} | {c}')
    
    state = initial_state(x)
    
    print(f'находим длину последовательности для x = {x}...')
    print()
    print_head()
    print_bar()
    print_state(0, state, "поехали!")
    for t, (i, state) in enumerate(run(state, program), 1):
        cmd, c = program[i]
        print_state('', cmd, c)
        print_bar()
        print_state(t, state, '')
    print_bar()
    
    answer = get_answer(state)
    print()
    print(f'L({x}) = {answer} за {t} тактов')
    return answer

In [6]:
run_kollatz(27, the_program)

находим длину последовательности для x = 27...

шаг  |   x   |   n   |   y   |   v1  |   v2  | комментарий
-----+-------+-------+-------+-------+-------+------------
   0 |    27 |     0 |     0 |     0 |     0 | поехали!
     |   -16 |     0 |     0 |     0 |    16 | START x >= 16
-----+-------+-------+-------+-------+-------+------------
   1 |    11 |     0 |     0 |     0 |    16 | 
     |    -2 |     0 |     8 |     3 |   -16 | x > 16+1
-----+-------+-------+-------+-------+-------+------------
   2 |     9 |     0 |     8 |     3 |     0 | 
     |    -2 |     0 |     1 |    -3 |     3 | DIV x/2 ...
-----+-------+-------+-------+-------+-------+------------
   3 |     7 |     0 |     9 |     0 |     3 | 
     |    -2 |     0 |     1 |     3 |    -3 | DIV x/2 ...
-----+-------+-------+-------+-------+-------+------------
   4 |     5 |     0 |    10 |     3 |     0 | 
     |    -2 |     0 |     1 |    -3 |     3 | DIV x/2 ...
-----+-------+-------+-------+-------+-------+----------

112